In [1]:
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import json
import pickle

## Settings here!
frame_size = 10
batch_size = 1000
## End settings

tqdm.pandas()

cuda = torch.device('cpu')
ratings = pd.read_csv('../../data/ml-20m/ratings.csv')
movies = pickle.load(open('../../data/infos_pca128.pytorch', 'rb'))

# credits: KnightofK9
ratings["rating"] = ratings["rating"].apply(lambda i: 2 * (i - 2.5))
users = ratings[["userId","movieId"]].groupby(["userId"]).size()
users = users[users >= frame_size + 1]

In [2]:
import h5py
import numpy as np
import torch
import pickle

# h5 = h5py.File("../../data/test/test.hdf5", "w")
idx = 0
n_batch = 0

optimize_ids = True
if optimize_ids:
    movie_ref = pickle.load(open('../../data/infos_pca128.pytorch', 'rb'))
    movie_ref[0] = torch.zeros(128)
    keys = list(sorted(movie_ref.keys()))
    key_to_id = dict(zip(keys, range(len(keys))))
    id_to_key = dict(zip(range(len(keys)), keys))
    ratings['movieId'] = ratings['movieId'].progress_apply(key_to_id.get)

In [7]:
ratings = ratings.sort_values(by='timestamp').set_index("userId").drop("timestamp", axis=1).groupby('userId')

for i in movies.keys():
    movies[i] = movies[i].to(cuda)

In [8]:
user_dict = {}
def app(x):
    userid = x.index[0]
    if userid in user_dict:
        print(userid)
        
    user_dict[int(userid)] = x.copy()
    # return x

In [9]:
ratings.progress_apply(app)
print()

1




In [9]:
for userid in tqdm(user_dict.keys()):
    x = user_dict[userid]
    grp = h5.create_group('u' + str(userid))
    grp.create_dataset('movies', data=x['movieId'].values.astype(np.int32))
    grp.create_dataset('ratings', data=x['rating'].values.astype(np.int8))

NameError: name 'h5' is not defined

In [10]:
# optional
# torch multithread dataloader doesnt work with hdf5
# you can save it as pickle dict 
# it is fully competible with the hdf5 dataloader, just change the file!
# p.s. dont forget to use n_workers > 1

user_dict_hdf5like = {}

for userid in tqdm(user_dict.keys()):
    x = user_dict[userid]
    user_dict_hdf5like['u' + str(userid)] = {}
    user_dict_hdf5like['u' + str(userid)]['movies'] = x['movieId'].values.astype(np.int32)
    user_dict_hdf5like['u' + str(userid)]['ratings'] = x['rating'].values.astype(np.int8)

In [11]:
pickle.dump( user_dict_hdf5like, open( "../../data/test/test.pkl", "wb" ) )

In [12]:
user_dict_hdf5like['u1']

{'movies': array([ 908,  903, 2598, 1533, 1058,  646, 2874,  334, 1276, 3903,  150,
         111, 1344, 1219, 1340, 2207, 4212, 1189, 1251, 1194, 1231, 6144,
        4626, 2631, 2563, 7861, 1914, 6393, 1319, 1293, 6646, 2204, 1190,
        1911, 2035, 6890, 4012, 5581, 1234, 5051, 3388, 2995, 2579, 1321,
        7276, 3840, 4133, 3179, 4783, 2944, 1176, 2915, 7345,   50,  538,
        1114, 5854, 1174, 7042, 1197, 1177,  588, 1068,   29, 4898, 6698,
        1170,  291,   47, 1235, 1172, 2607, 2676,  294, 7313, 8008,  316,
        1192, 1183,   32,  258, 1357, 3410, 4035,  251, 4659, 1305, 2560,
        2677, 1239, 1059, 1222, 1213, 3918, 2719, 1329, 2859, 1232, 2458,
        1069, 3934, 2833, 1018, 2111, 7954, 6235, 6430, 1264,  584, 2951,
        1216,  221, 2862, 8279, 7770, 2787, 1076, 2057, 5995, 5718, 4801,
        3391, 1884, 9758, 2110, 7840, 6665, 2055, 3067, 7456, 4373, 2883,
        6934, 2091, 4931, 5699, 4477, 2090, 4944, 5444, 7053, 1938, 6935,
        2060, 2017, 7136, 48